In [1]:
import pandas as pd
from glob import glob
import os
import geopandas as gpd
import re
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 130)
df = pd.read_csv("meta.csv")
df = df[["filename", "matched_image", "match_score", "res"]]
df["Pixel_ER"] = df.res.str.replace("(", "", regex=False).str.split(",").str[0].astype(float)
df.drop(columns="res", inplace=True)
df

,filename,matched_image,match_score,Pixel_ER
0,MaxarImagery/HighFreq/HawkesBay/Mahanga/Shorelines/Mahanga_31AUG2005.shp,MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif,100.0,0.600000
1,Retrolens/Tasman/ TorrentBay/Shorelines/TorrentBay_07NOV1952.shp,Retrolens/Tasman/ TorrentBay/Stack/TorrentBay_07NOV1952_mosaic.jp2,100.0,0.630361
2,Retrolens/Tasman/ TorrentBay/Shorelines/TorrentBay_26JAN1974.shp,Retrolens/Tasman/ TorrentBay/Stack/TorrentBay_26JAN1974_mosaic.jp2,100.0,0.676111
3,Retrolens/Tasman/ TorrentBay/Shorelines/TorrentBay_21FEB2002.shp,Retrolens/Tasman/ TorrentBay/Stack/TorrentBay_21FEB2002_mosaic.jp2,100.0,1.324992
4,Retrolens/Tasman/ TakakaRiver_North/Shorelines/TakakaRiver_31JAN1980.shp,Retrolens/Tasman/ TakakaRiver_North/Stack/TakakaRiver_31JAN1980_mosaic.jp2,100.0,1.397075
...,...,...,...,...
1346,MaxarImagery/HighFreq/Gisborne/TeAraroa/Shorelines/TeAraroa_22JAN2020.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_22JAN2020.tif,100.0,0.379230
1347,MaxarImagery/HighFreq/Gisborne/TeAraroa/Shorelines/TeAraroa_04MAY2004.shp,MaxarImagery/HighFreq/Gisborne/TeAraroa/Imagery/Stack/TeAraroa_04MAY2004.tif,100.0,0.600000
1348,MaxarImagery/HighFreq/Gisborne/Tolaga_KaiauaBay/Shorelines/Tolaga_KaiauaBay_22MAR2003.shp,MaxarImagery/HighFreq/Gisborne/Tolaga_KaiauaBay/Imagery/Stack/Tolaga_KaiauaBay_22MAR2003.tif,100.0,0.600000
1349,MaxarImagery/HighFreq/Gisborne/Wainui/Shorelines/Wainui_04JAN2004.shp,MaxarImagery/HighFreq/Gisborne/Wainui/Imagery/Stack/Wainui_04JAN2004.tif,100.0,0.600000


In [2]:
retro = df[df.filename.str.startswith("Retrolens")]
retro = retro[retro.filename.str.contains("BigBay_02APRL1953")]
retro

,filename,matched_image,match_score,Pixel_ER
1113,Retrolens/Southland/BigBay/Shorelines/BigBay_02APRL1953.shp,Retrolens/Southland/BigBay/Stack/BigBay_02APRL1953_mosaic.jp2,100.0,0.621867


In [3]:
shapefile = gpd.read_file(retro.filename.iloc[0])
shapefile

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,Proxy,geometry
0,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,3,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1208542.227 5079629.664, 1208548.577 5079633.632, 1208566.039 5079620.932, 1208586.677 5079614.582, 1208613.664 50..."
1,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,5,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1209199.797 5080178.490, 1209187.891 5080188.412, 1209175.588 5080195.556, 1209153.363 5080213.018, 1209146.616 50..."
2,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,4,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1210202.063 5083043.106, 1210201.071 5083051.044, 1210204.047 5083061.958, 1210203.386 5083071.549, 1210205.039 50..."
3,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,3,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1210157.150 5084697.510, 1210170.379 5084683.222, 1210187.841 5084680.047, 1210226.471 5084667.347, 1210256.633 50..."


In [4]:
def get_scale(filename):
    bits = filename.split("/")
    terminator = min(bits.index("Stack") if "Stack" in bits else 1024, bits.index("Shorelines") if "Shorelines" in bits else 1024)
    bits = bits[:terminator]
    csv_candidates = glob(f"ressci201900060-RNC2-Coastal/{'/'.join(bits)}/*.csv")
    assert len(csv_candidates) == 1
    csv = pd.read_csv(csv_candidates[0], encoding='cp1252')
    year = re.search(r'(\d{4})', filename).group(1)
    scales = csv[csv.Date.str.contains(year)].Scale.unique()
    assert len(scales) == 1
    return scales[0]

retro["Photoscale"] = retro.filename.apply(get_scale)
retro

,filename,matched_image,match_score,Pixel_ER,Photoscale
1113,Retrolens/Southland/BigBay/Shorelines/BigBay_02APRL1953.shp,Retrolens/Southland/BigBay/Stack/BigBay_02APRL1953_mosaic.jp2,100.0,0.621867,18200


In [5]:
def get_Georef_ER(scale):
  if scale < 20000:
    return 2.09
  elif scale < 30000:
    return 2.43
  else:
    return 2.9
retro["Georef_ER"] = retro.Photoscale.apply(get_Georef_ER)
retro

,filename,matched_image,match_score,Pixel_ER,Photoscale,Georef_ER
1113,Retrolens/Southland/BigBay/Shorelines/BigBay_02APRL1953.shp,Retrolens/Southland/BigBay/Stack/BigBay_02APRL1953_mosaic.jp2,100.0,0.621867,18200,2.09


In [6]:
Ep = retro.Pixel_ER.iloc[0]
Eg = retro.Georef_ER.iloc[0]
Ep, Eg

(0.6218672472343418, 2.09)

$E_t = \sqrt{E_p^2 + E_g^2 + E_d^2}$

In [7]:
shapefile

,Id,Region,Site,Date,DSASDate,Digitiser,Scale,Notes,Source,CPS,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,Proxy,geometry
0,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,3,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1208542.227 5079629.664, 1208548.577 5079633.632, 1208566.039 5079620.932, 1208586.677 5079614.582, 1208613.664 50..."
1,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,5,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1209199.797 5080178.490, 1209187.891 5080188.412, 1209175.588 5080195.556, 1209153.363 5080213.018, 1209146.616 50..."
2,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,4,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1210202.063 5083043.106, 1210201.071 5083051.044, 1210204.047 5083061.958, 1210203.386 5083071.549, 1210205.039 50..."
3,0,Southland,BigBay,1953-04-02,2/04/1953,MT,2000,NaN,RL,3,18200.0,2.09,0.62,2.38608,1,"LINESTRING (1210157.150 5084697.510, 1210170.379 5084683.222, 1210187.841 5084680.047, 1210226.471 5084667.347, 1210256.633 50..."


In [8]:
CPS_error_lookup = {
  1: .43,
  2: .73,
  3: .97,
  4: 2.07,
  5: 8.59
}
for i, row in shapefile.iterrows():
  Ed = CPS_error_lookup[row.CPS]
  Et = math.sqrt(Ep**2 + Eg**2 + Ed**2)
  print(f"√({Ep}² + {Eg}² + {Ed}²) = {Et}")

√(0.6218672472343418² + 2.09² + 0.97²) = 2.38657052549947
√(0.6218672472343418² + 2.09² + 8.59²) = 8.862444294503792
√(0.6218672472343418² + 2.09² + 2.07²) = 3.006612524616835
√(0.6218672472343418² + 2.09² + 0.97²) = 2.38657052549947
